In [4]:
#from ring_environment import Environment
import os
import gzip
import math
import copy
import pickle
import random
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp

from multiprocessing import Queue, Lock

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from agent import Agent

In [69]:
class Environment:
    def __init__(self, size, num_actions = 2, name = None, 
            max_iteration = 5000, num_agents = 20):
       
        node_num = self.size = size
        self.graph = nx.cycle_graph(node_num)
        self.nodes = list(self.graph.nodes)
        self.num_agents = num_agents

        #to do: sort mind function:
        #- inputs, outputs, handling each agent's individual mind

        #self.A_mind = Mind(input_size, num_actions, self.lock, Queue())
        #self.B_mind = Mind(input_size, num_actions, self.lock, Queue())
        #for segregation the agent_range defines the window that each agent can observe which is then input to that nn
        self.max_iteration = max_iteration

        #self.vals = [-1, 0, 1, 2]  #possible values for grid squares in segregation case
        #self.names_to_vals = {"void": -2, "A": -1, "free": 0, "B": 1, "prey": 2}
        #self.vals_to_names = {v: k for k, v in self.names_to_vals.items()}
        #self.vals_to_index = {-1: 0, 0: 1, 1: 2, 2: 3}

        self.crystal = np.zeros((max_iteration, node_num, 1))  #id  -should there be other info in this?
        #what actually is this crystal object? Is this what's being saved?

        self.history = []        #think about what quantities I want to save
        self.id_track = []       #how to define, intitialise and store them 
        self.records = []

        if name:
            self.name = name

        if not os.path.isdir(str(self.name)):
            #os.mkdir(str(self.name))
            #os.mkdir(str(self.name)+'/episodes')
            self.map, self.agents, self.loc_to_agent, self.id_to_agent = self._generate_map()
            #self._set_initial_states()
            #self.mask = self._get_mask()
            #self.crystal = np.zeros((max_iteration, nodes, 2)) # tr,  id
            #self.iteration = 0
        else:
            assert False, "There exists an experiment with this name."



    def _generate_map(self):
        #generates representation of environment in a given state
        #rewrite this function to give graph with a given number of agents on each node
        #where:
        # map = graph with n agents on each node (state of environment)
        # agents = agents
        # loc_to_agent = location of each agent   what is the structure of these dicionaries? 
        # id_to_agent = id of each agent
        # work out how to include mind networks
        map = np.zeros(self.size)   #index corresponds to graph node i.e. map[0] = 1 means there is one agent on node zero
        loc_to_agent = {}
        id_to_agent = {}
        agents = []
        idx = 0
        init_nodes = np.zeros(self.num_agents) #initialise agent locations (gives initial node loaction for each agent i.e. init_node[0] = 10 means agent 0 begins on node 10) 
        for j in range(0, self.num_agents):   #needs to be outside loop over nodes so doesn't change each time
            init_nodes[j] = np.random.choice(self.nodes)
            agent = Agent(idx, (init_nodes[j]))
            loc_to_agent[(init_nodes[j])] = agent
            id_to_agent[idx] = agent
            agents.append(agent)
            idx += 1

        for i in enumerate(map):
            #print(i)
            for x in range(0, self.num_agents):
                #print(i[1], init_nodes[x])
                if i[0] == init_nodes[x]:
                    
                    #print(int(i[1]))
                    map[i[0]] += 1
            
            #val = np.random.choice(self.vals, p=self.probs)   #val = number of agents on node i
                                                                #not a random choice - should this come after or within agent loop/assignation
                                                                #similar structure if val not == 0?
                                                                #how to assign number of agents to graph originally?

            #map[i] = val     #this is the 'value' of the grid square in the segregation problem being added to the map
                                    #this is what I need to change to graph nodes with given numbers of agents present
        return map, agents, loc_to_agent, id_to_agent


In [70]:
env = Environment(100, 2, 'Test1', 5000, 20)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [71]:
#map, agents, loc_to_agent, id_to_agent = env._generate_map()

In [72]:

print('map', env.map)

map [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 2. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]


In [75]:
print('agents', env.agents[0].get_loc())   #env.agents[3].get_id() returns 3 which is correct

#need to update location so that it is inputting actual agent location not the same number

agents 15.0


In [76]:
print('loc', env.loc_to_agent)

loc {15.0: <agent.Agent object at 0x7f09f47965c0>, 53.0: <agent.Agent object at 0x7f09f4797040>, 62.0: <agent.Agent object at 0x7f09f47955a0>, 43.0: <agent.Agent object at 0x7f09f47961d0>, 37.0: <agent.Agent object at 0x7f09f4797520>, 86.0: <agent.Agent object at 0x7f09f47965f0>, 89.0: <agent.Agent object at 0x7f09f47959f0>, 51.0: <agent.Agent object at 0x7f09f4796bc0>, 28.0: <agent.Agent object at 0x7f09f4797550>, 48.0: <agent.Agent object at 0x7f09f4795b40>, 31.0: <agent.Agent object at 0x7f09f4795840>, 3.0: <agent.Agent object at 0x7f09f4795150>, 1.0: <agent.Agent object at 0x7f09f4797fa0>, 94.0: <agent.Agent object at 0x7f09f4796bf0>, 85.0: <agent.Agent object at 0x7f09f4795570>, 40.0: <agent.Agent object at 0x7f09f4797580>, 10.0: <agent.Agent object at 0x7f09f4797f70>, 61.0: <agent.Agent object at 0x7f09f4797f40>}


In [44]:
print('id', env.id_to_agent)

id {0: <agent.Agent object at 0x7f09f57aaa10>, 1: <agent.Agent object at 0x7f09f5d7a9e0>, 2: <agent.Agent object at 0x7f09f5d7ab00>, 3: <agent.Agent object at 0x7f09f5d79c60>, 4: <agent.Agent object at 0x7f09f5d7ad10>, 5: <agent.Agent object at 0x7f09f5d78a00>, 6: <agent.Agent object at 0x7f09f5d7a8f0>, 7: <agent.Agent object at 0x7f09f5d7b0d0>, 8: <agent.Agent object at 0x7f09f5d7add0>, 9: <agent.Agent object at 0x7f09f5d7a980>, 10: <agent.Agent object at 0x7f09f5d789d0>, 11: <agent.Agent object at 0x7f09f5d79990>, 12: <agent.Agent object at 0x7f09f5d7ad40>, 13: <agent.Agent object at 0x7f09f5d7a440>, 14: <agent.Agent object at 0x7f09f5d7af80>, 15: <agent.Agent object at 0x7f09f5d7b040>, 16: <agent.Agent object at 0x7f09f5d7af50>, 17: <agent.Agent object at 0x7f09f5d7ace0>, 18: <agent.Agent object at 0x7f09f5d7aa10>, 19: <agent.Agent object at 0x7f09f5d7ae60>}


In [77]:
#now know what all of these outputs are I can move onto other functions tomorrow 